In [1]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, losses
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, Cropping2D

import pandas as pd
from keras.utils import np_utils

from PIL import Image
import numpy as np

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [53]:
tdata = pd.read_csv('125_train_concepts.txt', delimiter=":", header=None)
vdata = pd.read_csv('125_validation_concepts.txt', delimiter=":", header=None)

In [54]:
tdata.head()

,0,1
0,C0441633,pone.0108478.g001;OJO-9-59-g003;1746-160X-1-6-...
1,C0729619,IJPsy-57-91-g001;IJPsy-57-91-g002;JCIS-3-61-g0...
2,C0412621,IJPsy-57-91-g001;IJPsy-57-91-g002;JCIS-3-61-g0...
3,C0772294,jod-8-152f1;IJD-56-568-g005;IJD-56-568-g012;18...
4,C0412643,IJPsy-57-91-g001;IJPsy-57-91-g002;JCIS-3-61-g0...


In [80]:
tres = []
for index, row in tdata[:3].iterrows():
#     print index
    for i in row[1].split(';'):
#         print str(i)+';'+str(row[0])+';'+str(index)
        tres.append(str(i)+';'+str(row[0])+';'+str(index))  
    
vres = []
for index, row in vdata[:3].iterrows():
#     print index
    for i in row[1].split(';'):
#         print str(i)+';'+str(row[0])+';'+str(index)
        vres.append(str(i)+';'+str(row[0])+';'+str(index)) 

In [81]:
train_df = pd.DataFrame(
    {
        "0":tres,
    })
print len(train_df)

val_df = pd.DataFrame(
    {
        "0":vres,
    })
print len(val_df)

11122
3708


In [82]:
train_df.to_csv('125Ctrain.csv',  index = False, header=False)
val_df.to_csv('125Cval.csv',  index = False, header=False)

In [83]:
from sklearn.utils import shuffle


now = pd.read_csv('/Users/sharath/Desktop/GP/sharath/125Ctrain.csv', delimiter=";", header=None)
k = now[[2]].values
res = np_utils.to_categorical(k)
leng =  res.shape[1]
dataset_size  = len(now[0])

labels = pd.DataFrame(res)

result = pd.concat([now, labels], axis=1, join_axes=[now.index])
# result = shuffle(result)
print result.head()

result.to_csv("preproccesed_train.txt", header=False)


                   0         1  2    0    1    2
0  pone.0108478.g001  C0441633  0  1.0  0.0  0.0
1      OJO-9-59-g003  C0441633  0  1.0  0.0  0.0
2    1746-160X-1-6-3  C0441633  0  1.0  0.0  0.0
3  2052-336X-12-56-3  C0441633  0  1.0  0.0  0.0
4     JCIS-3-61-g007  C0441633  0  1.0  0.0  0.0


In [84]:
Vnow = pd.read_csv('/Users/sharath/Desktop/GP/sharath/125Cval.csv', delimiter=";", header=None)
k = Vnow[[2]].values
Vres = np_utils.to_categorical(k)
Vleng =  Vres.shape[1]
# dataset_size  = len(now[0])

Vlabels = pd.DataFrame(Vres)

Vresult = pd.concat([Vnow, Vlabels], axis=1, join_axes=[Vnow.index])
# result = shuffle(result)
print Vresult.head()

result.to_csv("preproccesed_Val.txt", header=False)


                    0         1  2    0    1    2
0      gr2_PMC4983645  C0441633  0  1.0  0.0  0.0
1     PAMJ-25-65-g001  C0441633  0  1.0  0.0  0.0
2  rt-2011-2-e18-g001  C0441633  0  1.0  0.0  0.0
3  rt-2011-2-e18-g002  C0441633  0  1.0  0.0  0.0
4        srep23711-f4  C0441633  0  1.0  0.0  0.0


In [85]:
imgdim = 80

In [86]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    
    return 2*((precision*recall)/(precision+recall))

# import keras
# class Metrics(keras.callbacks.Callback):
#     def on_epoch_end(batch, logs={}):
#         predict = np.asarray(model.predict(validation_data[0]))
#         targ = self.validation_data[1]
#         f1s=f1(targ, predict)
#         return
# metrics = Metrics()


In [87]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape = (imgdim, imgdim,3),border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(leng, activation = 'softmax')) 

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=[f1])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 80, 80, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               25600500  
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 1503      
Total params: 25,602,899
Trainable params: 25,602,899
Non-trainable params: 0
_________________________________________________________________


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(80, 80, 3...)`
  


In [88]:
def Train_gen(path):
    
    def process_line(row):
        temp =  row.split(',')
#         print temp[0]
        return temp[1:2], temp[4:]
    def load_images(img):
#         print img
        img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ img[0] +".jpg")
        img = img.resize((imgdim,imgdim)).convert('RGB')
        T = (np.expand_dims(img,axis=0))
        return T

        
    while 1:
        f = open(path)
        for line in f:
            x,y = process_line(line)
            img = load_images(x)
            y = np.expand_dims(y,axis=0)
#             print img.shape, y.shape
            yield (img, y)
        f.close()

        
def Val_gen(path):
    
    def process_line(row):
        temp =  row.split(',')
#         print temp[0]
        return temp[1:2], temp[4:]
    def load_images(img):
#         print img
        img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ img[0] +".jpg")
        img = img.resize((imgdim,imgdim)).convert('RGB')
        T = (np.expand_dims(img,axis=0))
        return T

        
    while 1:
        f = open(path)
        for line in f:
            x,y = process_line(line)
            img = load_images(x)
            y = np.expand_dims(y,axis=0)
#             print img.shape, y.shape
            yield (img, y)
        f.close()
        
tpath = '/Users/sharath/Desktop/GP/sharath/preproccesed_train.txt'
vpath = '/Users/sharath/Desktop/GP/sharath/preproccesed_Val.txt'

    
        


In [89]:
model.fit_generator(Train_gen(tpath) ,steps_per_epoch=dataset_size, epochs=1, validation_data=Val_gen(vpath),
                    validation_steps=dataset_size)

Epoch 1/1
  411/11122 [>.............................] - ETA: 1:26:43 - loss: 3.8491e-07 - f1: 1.0000

KeyboardInterrupt: 

In [19]:
T = []
name = 'aps-43-590-g001'
img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ name +".jpg")
img = img.resize((imgdim,imgdim)).convert('RGB')
T.append(np.array(img))
X = np.array(T)
X.shape

(1, 64, 64, 3)

In [20]:
model.predict(X)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
def test_genarator(track_index):
    track_index = track_index
    X = []
    y = []
    print  track_index
    for index, row in testdata[track_index:track_index+plus_track_index].iterrows():
#         print row.index
        x = row.values[0]
        label = row.values[3:]
        x = row.values[0]
        y.append(label)
        img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ x +".jpg")
        img = img.resize((imgdim,imgdim)).convert('RGB')
        T = (np.array(img))
        X.append(T)
#         track_index =  index
    X_train = np.array(X)
    y_train = np.array(y)
#     print X_train.shape, y_train.shape
    return X_train, y_train




In [ ]:
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(x_train)

# # # fits the model on batches with real-time data augmentation:
# # model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
# #                     steps_per_epoch=len(x_train) / 32, epochs=epochs)

# # here's a more "manual" example
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break


In [ ]:
 # def generate_arrays_from_file():
    
#     while 1:
#         f = open(path)
#         for line in f:
#             # create numpy arrays of input data
#             # and labels, from each line in the file
#             x, y = process_line(line)
#             img = load_images(x)
#             yield (img, y)
#         f.close()

        
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='10',epochs=10)


# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')

In [1]:


# def generate_arrays_from_file():
#     for index, row in now.iterrows():
#         print index
#         x = row.values[0]
#         y = res[index]
#         img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ x +".jpg")
#         T.append(np.array(img))
#         X = np.array(T)
#     yield (X, y)
    
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='1',epochs=1)






# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')